<a href="https://www.kaggle.com/code/nguynvnln22028281/btl-nlp-cleandata?scriptVersionId=286613002" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
!pip install langid sentence-transformers


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from datasets import load_dataset

train_dataset = load_dataset("ncduy/mt-en-vi", split = "train")

valid_dataset = load_dataset("ncduy/mt-en-vi", split = "validation")

test_dataset = load_dataset("ncduy/mt-en-vi", split = "test")



In [ ]:
train_dataset = train_dataset.shuffle(seed=42).select(range(1000000))
print(len(train_dataset))

In [ ]:
from torch.utils.data import Dataset
from tqdm import tqdm
import re
import unicodedata
import langid
import torch

# ENABLE semantic filtering for medical data
use_semantic_filter = False
similarity_threshold = 0.70  # Raised for technical content

if use_semantic_filter:
    from sentence_transformers import SentenceTransformer, util
    print("Loading LaBSE model...")
    labse = SentenceTransformer("sentence-transformers/LaBSE")
    device = "cuda" if torch.cuda.is_available() else "cpu"
    labse = labse.to(device)

# ---------------------------
# Enhanced Cleaning Functions
# ---------------------------

def normalize_text(s):
    # Remove HTML/XML
    s = re.sub(r"<[^>]+>", "", s)

    # Unicode normalize (this is SAFE)
    s = unicodedata.normalize("NFC", s)

    # Remove zero-width chars
    s = s.replace("\u200b", "").replace("\ufeff", "")

    # Fix spacing
    s = re.sub(r'\s+([.,;:!?])', r'\1', s)
    s = re.sub(r"\s+", " ", s).strip()

    return s

def is_metadata_line(s):
    """Detect non-content lines (headers, page numbers, etc.)"""
    s_lower = s.lower()
    
    # Check for common metadata patterns
    metadata_patterns = [
        r'^page \d+$',
        r'^\d+\s*$',  # Just numbers
        r'^(abstract|introduction|conclusion|references?|methods?):\s*$',
        r'^\w+\s+\d{4}$',  # "December 2021"
        r'^volume \d+',
        r'^doi:',
        r'^issn',
        r'^copyright',
    ]
    
    for pattern in metadata_patterns:
        if re.match(pattern, s_lower):
            return True
    
    # Too short to be meaningful content
    if len(s.split()) < 4:
        return True
        
    return False

def heuristic_bad_pair(en, vi):
    """Enhanced domain-specific filters"""
    en_low = en.lower()
    vi_low = vi.lower()
    
    # Known translation errors in this dataset
    if "vaginal" in en_low and any(kw in en_low for kw in ["ear", "otitis", "tympanogram"]):
        return True
    
    # Corrupted spellings
    if any(bad in en_low for bad in ["otittis", "rhinolaryngology", "imumnohistochemistry"]):
        return True
    
    # Detect if one side is metadata but other isn't
    if is_metadata_line(en) != is_metadata_line(vi):
        return True
    
    # Both are metadata
    if is_metadata_line(en) and is_metadata_line(vi):
        return True
    
    # Check for URL/email mismatches
    en_has_url = bool(re.search(r'https?://|www\.', en))
    vi_has_url = bool(re.search(r'https?://|www\.', vi))
    if en_has_url != vi_has_url:
        return True
    
    # Detect if one sentence has numbers/stats but other doesn't
    en_has_nums = bool(re.search(r'\d+[.,]?\d*\s*[%±]', en))
    vi_has_nums = bool(re.search(r'\d+[.,]?\d*\s*[%±]', vi))
    # Allow medical texts with stats on one side (could be rephrased)
    # But reject if one has LOTS of numbers and other has none
    en_num_count = len(re.findall(r'\d+', en))
    vi_num_count = len(re.findall(r'\d+', vi))
    if en_num_count > 5 and vi_num_count == 0:
        return True
    if vi_num_count > 5 and en_num_count == 0:
        return True
    
    return False

def length_ratio_bad(en, vi):
    """Stricter length constraints for medical abstracts"""
    len_en, len_vi = len(en.split()), len(vi.split())
    
    # Minimum length (medical sentences are usually substantial)
    if len_en < 3 or len_vi < 3:
        return True
    
    # Maximum length (likely concatenated paragraphs)
    if len_en > 150 or len_vi > 150:
        return True
    
    # Tighter ratio for technical content
    ratio = len_en / max(len_vi, 1)
    if ratio > 2 or ratio < 0.5:
        return True
    return False

def language_mismatch(en, vi):
    """Relaxed language detection for medical text"""
    # Medical text may be unreliable for langid - use as soft signal
    try:
        lang_en, score_en = langid.classify(en)
        lang_vi, score_vi = langid.classify(vi)
        
        # Only reject on very confident misdetections
        if score_en > 0.9 and lang_en != "en":
            return True
        if score_vi > 0.9 and lang_vi != "vi":
            return True
            
    except Exception:
        # If langid fails, don't reject
        pass
    
    # Additional heuristic: Vietnamese should have tone marks
    vietnamese_chars = set("áàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệíìỉĩịóòỏõọôốồổỗộơớờởỡợúùủũụưứừửữựýỳỷỹỵđ")
    vi_lower = vi.lower()
    has_vietnamese_chars = any(c in vietnamese_chars for c in vi_lower)
    
    # Reject if supposed Vietnamese has zero tone marks (likely English)
    if len(vi_lower) > 20 and not has_vietnamese_chars:
        return True
    
    return False

def semantic_bad(en, vi):
    """Compute semantic similarity with batching for speed"""
    with torch.no_grad():
        emb_en = labse.encode(en, convert_to_tensor=True, device=device)
        emb_vi = labse.encode(vi, convert_to_tensor=True, device=device)
        score = util.cos_sim(emb_en, emb_vi).item()
    return score < similarity_threshold


# ---------------------------
# Dataset Class
# ---------------------------
class ParallelTextDataset(Dataset):
    def __init__(self, hf_dataset):
        self.data = []
        seen = set()
        rejected_length_samples = []
        
        
        stats = {
            'total': 0,
            'empty': 0,
            'heuristic': 0,
            'length': 0,
            'language': 0,
            'semantic': 0,
            'duplicate': 0,
            'kept': 0
        }

        for item in tqdm(hf_dataset, desc="CLeaning HF dataset"):
            src = item["en"].strip()
            tgt = item["vi"].strip()
            stats["total"] += 1

            if not src or not tgt:
                stats["empty"] += 1
                continue
                
            #normalize
            en = normalize_text(src)
            vi = normalize_text(tgt)

            if not en or not vi:
                stats["empty"] += 1
                continue
                
            # Filter pipeline
            if heuristic_bad_pair(en, vi):
                stats['heuristic'] += 1
                continue
            
            if length_ratio_bad(en, vi):
                if len(rejected_length_samples) < 20:  # Collect 20 samples
                    rejected_length_samples.append({
                        'en': en,
                        'vi': vi,
                        'len_en': len(en.split()),
                        'len_vi': len(vi.split()),
                        'ratio': len(en.split()) / max(len(vi.split()), 1)
                    })
                stats['length'] += 1
                continue
            
            if language_mismatch(en, vi):
                stats['language'] += 1
                continue
            
            if use_semantic_filter and semantic_bad(en, vi):
                stats['semantic'] += 1
                continue
            
            # Deduplicate
            key = en + "|||" + vi
            if key in seen:
                stats['duplicate'] += 1
                continue
            
            seen.add(key)
            self.data.append((en, vi))
            stats['kept'] += 1
    
        # Print detailed statistics
        print(f"\n{'='*60}")
#        print(f"Dataset: {src_file.split('/')[-1]}")
        print(f"{'='*60}")
        print(f"Total pairs processed:     {stats['total']:>6}")
        print(f"  - Empty/blank:           {stats['empty']:>6} ({stats['empty']/stats['total']*100:>5.1f}%)")
        print(f"  - Heuristic filters:     {stats['heuristic']:>6} ({stats['heuristic']/stats['total']*100:>5.1f}%)")
        print(f"  - Length ratio:          {stats['length']:>6} ({stats['length']/stats['total']*100:>5.1f}%)")
        print(f"  - Language mismatch:     {stats['language']:>6} ({stats['language']/stats['total']*100:>5.1f}%)")
        if use_semantic_filter:
            print(f"  - Semantic similarity:   {stats['semantic']:>6} ({stats['semantic']/stats['total']*100:>5.1f}%)")
        print(f"  - Duplicates:            {stats['duplicate']:>6} ({stats['duplicate']/stats['total']*100:>5.1f}%)")
        print(f"{'='*60}")
        print(f"CLEAN PAIRS KEPT:          {stats['kept']:>6} ({stats['kept']/stats['total']*100:>5.1f}%)")
        print(f"{'='*60}\n")

        print("\n" + "="*60)
        print("SAMPLE REJECTED PAIRS (Length Ratio):")
        print("="*60)
        for i, sample in enumerate(rejected_length_samples[:10], 1):
            print(f"\n--- Sample {i} ---")
            print(f"EN ({sample['len_en']} words): {sample['en'][:150]}...")
            print(f"VI ({sample['len_vi']} words): {sample['vi'][:150]}...")
            print(f"Ratio: {sample['ratio']:.2f}")
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

In [ ]:
# ---------------------------
# Usage
# ---------------------------
train_dataset = ParallelTextDataset(train_dataset)

In [ ]:
valid_dataset = ParallelTextDataset(valid_dataset)


In [ ]:
test_dataset = ParallelTextDataset(test_dataset)


In [ ]:
# Test it
for i in range(10):
    src, tgt = test_dataset[i]
    print(f"Source: {src}")
    print(f"Target: {tgt}")

print(len(test_dataset))

In [ ]:
import json

def save_to_jsonl(dataset, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        for en, vi in dataset.data:
            json_line = json.dumps({"en": en, "vi": vi}, ensure_ascii=False)
            f.write(json_line + '\n')

save_to_jsonl(train_dataset, "train_cleaned.jsonl")
save_to_jsonl(valid_dataset, "valid_cleaned.jsonl")

save_to_jsonl(test_dataset, "test_cleaned.jsonl")